# Download PDS data products

In the previous/first step we have queried PDS/ODE API for the data products over a region (Mawrth Vallis) of Mars.
In particular, we search for data from the MRO/CTX, MRO/HiRISE, and MEX/HRSC instruments.
The results -- i.e., footprints and associated metadata -- were written in GeoJSON file(s).

We will go now through the process of (1) downloading the/some data products, and (2) updating the metadata set we have for each image. Naturally, we are going to the (NPT) library to do it.

In [1]:
search_results = 'Mawrth_Vallis_ODE_products_intersect.geojson'

In [2]:
import npt

# gjson = npt.utils.read_geojson(search_results)

In [3]:
import geopandas
import pandas
pandas.set_option('display.max_columns',100)

In [4]:
gdf = geopandas.read_file(search_results)

gdf_sample = gdf.groupby('inst').head(2).reset_index(drop=True)

gdf_sample

id mission    inst    type Target_name  \
0          H1293_0000_ND3.JP2     MEX    HRSC  REFDR3        MARS   
1          H2196_0001_ND3.JP2     MEX    HRSC  REFDR3        MARS   
2  P01_001388_2034_XI_23N017W     MRO     CTX     EDR        MARS   
3  P04_002641_2046_XN_24N017W     MRO     CTX     EDR        MARS   
4         PSP_001388_2035_RED     MRO  HIRISE  RDRV11        MARS   
5         PSP_002641_2025_RED     MRO  HIRISE  RDRV11        MARS   

  Footprints_cross_meridian Map_scale Center_latitude Center_longitude  \
0                     False      12.5         29.7761         341.8979   
1                     False        25         16.2676         344.0262   
2                     False      5.66         23.2688         342.4958   
3                     False      5.66         24.3758         342.2817   
4                     False      0.25         23.2452         342.5015   
5                     False      0.25         22.2517          342.551   

  Easternmost_longitude Westernmost_longitude Minimum_latitude  \
0                342.89               340.677           19.519   
1               345.891               342.568            8.632   
2                 342.8                342.19          22.8866   
3                342.87                341.66          21.7462   
4               342.563               342.439           23.161   
5               342.621               342.481          22.1027   

  Maximum_latitude Emission_angle Incidence_angle Phase_angle Solar_longitude  \
0           40.065           None            None        None            None   
1          23.9027           None            None        None            None   
2          23.6504           5.85           48.88       43.11          134.36   
3          27.0054           0.09           59.39       59.41          185.96   
4          23.3293       5.816564       48.900419   43.137054       134.35984   
5          22.4007       0.295027       58.737351   58.835998      185.953173   

             Observation_time Product_creation_time  \
0  2005-01-19T19:06:48.344002   2016-03-04T20:31:26   
1  2005-09-29T15:28:07.719000   2016-03-14T11:38:04   
2  2006-11-12T15:12:52.777000   2007-05-18T22:07:23   
3  2007-02-18T05:52:00.318000   2007-08-09T21:43:46   
4  2006-11-12T15:12:52.248001   2009-09-22T22:45:09   
5  2007-02-18T05:51:20.181000   2009-11-11T10:51:58   

               UTC_start_time               UTC_stop_time  \
0  2005-01-19T19:04:06.841000  2005-01-19T19:09:29.847000   
1  2005-09-29T15:25:45.728001  2005-09-29T15:30:29.709999   
2  2006-11-12T15:12:46.050999  2006-11-12T15:12:59.504002   
3  2007-02-18T05:51:11.313000  2007-02-18T05:52:49.324001   
4  2006-11-12T15:12:50.716999  2006-11-12T15:12:53.779999   
5  2007-02-18T05:51:17.450001  2007-02-18T05:51:22.912001   

                                           image_url image_kbytes  \
0  https://pds-geosciences.wustl.edu/mex/mex-m-hr...       286318   
1  https://pds-geosciences.wustl.edu/mex/mex-m-hr...        94483   
2  https://pds-imaging.jpl.nasa.gov/data/mro/mars...        35001   
3  https://pds-imaging.jpl.nasa.gov/data/mro/mars...       252001   
4  https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...       373388   
5  https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...       980973   

                                           label_url  \
0  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   
1  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   
2                                               None   
3                                               None   
4  https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...   
5  https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB...   

                                          browse_url  \
0  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   
1  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   
2                                               None   
3                                               None   
4  https://hirise.lpl.

## Overview

During the whole of the downloading process is when we will fix eventual gaps in our metadata set.
For instance, have a look at the sample table above -- we have been carefull to select a couple of entries from the different datasets (_instruments_) so we can have a general view of the dataset at hand.

We notice the following points that should be fixed/adjusted at the end of this process:

- HRSC `id` is literally the product filename: remove the file extension from it to better define it as ID;
- Longitude coordinates are in the range [0:360]: shift it to [-180:180] to be compatible with typical geo-spatial databases and tools;
- HRSC entries have no values for `Emission_angle`, `Incidence_angle`, `Phase_angle`, `Solar_longitude`: recover them from image headers/Label files;
- CTX products have no separated Label files: write them from the Label information in the image headers.

## Split data

To keep things simple and under control, we want to download data products for each instrument separately.

In [5]:
gdf_sample_ctx = gdf_sample.query('inst == "CTX"')

gdf_sample_hirise = gdf_sample.query('inst == "HIRISE"')

gdf_sample_hrsc = gdf_sample.query('inst == "HRSC"')

## Convert (DF) tables to GeoJSON

GeoJSON (features) -- i.e., Python dictionaries -- is the data structure used in NPT download function(s).

In [6]:
import json

gjson_ctx = json.loads(gdf_sample_ctx.to_json())

gjson_hirise = json.loads(gdf_sample_hirise.to_json())

gjson_hrsc = json.loads(gdf_sample_hrsc.to_json())

## CTX

In [7]:
import npt

In [8]:
gjson_ctx_new = npt.download.from_geojson(gjson_ctx, basepath='data/ctx', progressbar=True)

function _is_downloaded()
*url = https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0008/data/P01_001388_2034_XI_23N017W.IMG
*filename = data/ctx/P01_001388_2034_XI_23N017W.IMG
-return True
PDS LABEL file 'data/ctx/P01_001388_2034_XI_23N017W.lbl' created.
Feature/metadata file 'data/ctx/P01_001388_2034_XI_23N017W.json' written.
function _is_downloaded()
*url = https://pds-imaging.jpl.nasa.gov/data/mro/mars_reconnaissance_orbiter/ctx/mrox_0057/data/P04_002641_2046_XN_24N017W.IMG
*filename = data/ctx/P04_002641_2046_XN_24N017W.IMG
-return True
PDS LABEL file 'data/ctx/P04_002641_2046_XN_24N017W.lbl' created.
Feature/metadata file 'data/ctx/P04_002641_2046_XN_24N017W.json' written.


In [9]:
from geopandas import GeoDataFrame

gdf_ctx_new = GeoDataFrame.from_features(gjson_ctx_new['features'])
gdf_ctx_new

geometry Center_latitude  \
0  POLYGON ((-17.29000 23.65040, -17.20000 22.936...         23.2688   
1  POLYGON ((-17.80000 27.00540, -17.13000 21.805...         24.3758   

  Center_longitude Easternmost_longitude Emission_angle  \
0         342.4958                 342.8           5.85   
1         342.2817                342.87           0.09   

  Footprints_cross_meridian Incidence_angle Map_scale Maximum_latitude  \
0                     False           48.88      5.66          23.6504   
1                     False           59.39      5.66          27.0054   

  Minimum_latitude            Observation_time Phase_angle  \
0          22.8866  2006-11-12T15:12:52.777000       43.11   
1          21.7462  2007-02-18T05:52:00.318000       59.41   

  Product_creation_time Solar_longitude Target_name  \
0   2007-05-18T22:07:23          134.36        MARS   
1   2007-08-09T21:43:46          185.96        MARS   

               UTC_start_time               UTC_stop_time  \
0  2006-11-12T15:12:46.050999  2006-11-12T15:12:59.504002   
1  2007-02-18T05:51:11.313000  2007-02-18T05:52:49.324001   

  Westernmost_longitude browse_url                          id image_kbytes  \
0                342.19       None  P01_001388_2034_XI_23N017W        35001   
1                341.66       None  P04_002641_2046_XN_24N017W       252001   

  inst mission type                               image_path  \
0  CTX     MRO  EDR  data/ctx/P01_001388_2034_XI_23N017W.IMG   
1  CTX     MRO  EDR  data/ctx/P04_002641_2046_XN_24N017W.IMG   

                                label_path           instrumentHostName  \
0  data/ctx/P01_001388_2034_XI_23N017W.lbl  MARS RECONNAISSANCE ORBITER   
1  data/ctx/P04_002641_2046_XN_24N017W.lbl  MARS RECONNAISSANCE ORBITER   

   instrumentName missionPhaseName              idFromProvider  \
0  CONTEXT CAMERA              PSP  P01_001388_2034_XI_23N017W   
1  CONTEXT CAMERA              PSP  P04_002641_2046_XN_24N017W   

  observationMode instrumentId spacecraftId datasetId targetName  \
0             EDR          CTX          MRO   MRO_CTX       MARS   
1             EDR          CTX          MRO   MRO_CTX       MARS   

   productCreationTime                   startTime                    stopTime  
0  2007-05-18T22:07:23  2006-11-12T15:12:46.050999  2006-11-12T15:12:59.504002  
1  2007-08-09T21:43:46  2007-02-18T05:51:11.313000  2007-02-18T05:52:49.324001

In [10]:
gjson_hirise_new = npt.download.from_geojson(gjson_hirise, basepath='data/hirise', progressbar=True)

function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_001300_001399/PSP_001388_2035/PSP_001388_2035_RED.JP2
*filename = data/hirise/PSP_001388_2035_RED.JP2
-return True
function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_001300_001399/PSP_001388_2035/PSP_001388_2035_RED.LBL
*filename = data/hirise/PSP_001388_2035_RED.LBL
-return True
Feature/metadata file 'data/hirise/PSP_001388_2035_RED.json' written.
function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_002600_002699/PSP_002641_2025/PSP_002641_2025_RED.JP2
*filename = data/hirise/PSP_002641_2025_RED.JP2
-return True
function _is_downloaded()
*url = https://hirise.lpl.arizona.edu/PDS/RDR/PSP/ORB_002600_002699/PSP_002641_2025/PSP_002641_2025_RED.LBL
*filename = data/hirise/PSP_002641_2025_RED.LBL
-return True
Feature/metadata file 'data/hirise/PSP_002641_2025_RED.json' written.


In [11]:
gdf_hirise_new = GeoDataFrame.from_features(gjson_hirise_new['features'])
gdf_hirise_new

geometry Center_latitude  \
0  POLYGON ((-17.43700 23.17200, -17.54100 23.161...         23.2452   
1  POLYGON ((-17.37900 22.11410, -17.48200 22.102...         22.2517   

  Center_longitude Easternmost_longitude Emission_angle  \
0         342.5015               342.563       5.816564   
1          342.551               342.621       0.295027   

  Footprints_cross_meridian Incidence_angle Map_scale Maximum_latitude  \
0                     False       48.900419      0.25          23.3293   
1                     False       58.737351      0.25          22.4007   

  Minimum_latitude            Observation_time Phase_angle  \
0           23.161  2006-11-12T15:12:52.248001   43.137054   
1          22.1027  2007-02-18T05:51:20.181000   58.835998   

  Product_creation_time Solar_longitude Target_name  \
0   2009-09-22T22:45:09       134.35984        MARS   
1   2009-11-11T10:51:58      185.953173        MARS   

               UTC_start_time               UTC_stop_time  \
0  2006-11-12T15:12:50.716999  2006-11-12T15:12:53.779999   
1  2007-02-18T05:51:17.450001  2007-02-18T05:51:22.912001   

  Westernmost_longitude                                         browse_url  \
0               342.439  https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/...   
1               342.481  https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/...   

                    id image_kbytes    inst mission    type  \
0  PSP_001388_2035_RED       373388  HIRISE     MRO  RDRV11   
1  PSP_002641_2025_RED       980973  HIRISE     MRO  RDRV11   

                            image_path                           label_path  \
0  data/hirise/PSP_001388_2035_RED.JP2  data/hirise/PSP_001388_2035_RED.LBL   
1  data/hirise/PSP_002641_2025_RED.JP2  data/hirise/PSP_002641_2025_RED.LBL   

            instrumentHostName                              instrumentName  \
0  MARS RECONNAISSANCE ORBITER  HIGH RESOLUTION IMAGING SCIENCE EXPERIMENT   
1  MARS RECONNAISSANCE ORBITER  HIGH RESOLUTION IMAGING SCIENCE EXPERIMENT   

        missionPhaseName       idFromProvider observationMode instrumentId  \
0  PRIMARY SCIENCE PHASE  PSP_001388_2035_RED          RDRV11       HIRISE   
1  PRIMARY SCIENCE PHASE  PSP_002641_2025_RED          RDRV11       HIRISE   

  spacecraftId   datasetId targetName  productCreationTime  \
0          MRO  MRO_HIRISE       MARS  2009-09-22T22:45:09   
1          MRO  MRO_HIRISE       MARS  2009-11-11T10:51:58   

                    startTime                    stopTime  
0  2006-11-12T15:12:50.716999  2006-11-12T15:12:53.779999  
1  2007-02-18T05:51:17.450001  2007-02-18T05:51:22.912001

In [12]:
gjson_hrsc_new = npt.download.from_geojson(gjson_hrsc, basepath='data/hrsc', progressbar=True)

function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/1293/h1293_0000_nd3.jp2
*filename = data/hrsc/h1293_0000_nd3.jp2
-return True
function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/1293/h1293_0000_nd3.lbl
*filename = data/hrsc/h1293_0000_nd3.lbl
-return False
--> Downloading file data/hrsc/h1293_0000_nd3.lbl ..


data/hrsc/h1293_0000_nd3.lbl: 4KB [00:00, 600.86KB/s]                                                                                                                                


--> File downloaded.
Feature/metadata file 'data/hrsc/h1293_0000_nd3.json' written.
function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/2196/h2196_0001_nd3.jp2
*filename = data/hrsc/h2196_0001_nd3.jp2
-return True
function _is_downloaded()
*url = https://pds-geosciences.wustl.edu/mex/mex-m-hrsc-5-refdr-mapprojected-v3/mexhrs_1003/data/2196/h2196_0001_nd3.lbl
*filename = data/hrsc/h2196_0001_nd3.lbl
-return False
--> Downloading file data/hrsc/h2196_0001_nd3.lbl ..


data/hrsc/h2196_0001_nd3.lbl: 3KB [00:00, 1030.54KB/s]                                                                                                                               

--> File downloaded.
Feature/metadata file 'data/hrsc/h2196_0001_nd3.json' written.


In [13]:
gdf_hrsc_new = GeoDataFrame.from_features(gjson_hrsc_new['features'])
gdf_hrsc_new

geometry Center_latitude  \
0  POLYGON ((-17.82000 40.06500, -17.81000 39.882...         29.7761   
1  POLYGON ((-16.59400 8.63200, -16.60100 8.78700...         16.2676   

  Center_longitude Easternmost_longitude Emission_angle  \
0         341.8979                342.89           None   
1         344.0262               345.891           None   

  Footprints_cross_meridian Incidence_angle Map_scale Maximum_latitude  \
0                     False            None      12.5           40.065   
1                     False            None        25          23.9027   

  Minimum_latitude            Observation_time Phase_angle  \
0           19.519  2005-01-19T19:06:48.344002        None   
1            8.632  2005-09-29T15:28:07.719000        None   

  Product_creation_time Solar_longitude Target_name  \
0   2016-03-04T20:31:26            None        MARS   
1   2016-03-14T11:38:04            None        MARS   

               UTC_start_time               UTC_stop_time  \
0  2005-01-19T19:04:06.841000  2005-01-19T19:09:29.847000   
1  2005-09-29T15:25:45.728001  2005-09-29T15:30:29.709999   

  Westernmost_longitude                                         browse_url  \
0               340.677  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   
1               342.568  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   

                   id image_kbytes  inst mission    type  \
0  H1293_0000_ND3.JP2       286318  HRSC     MEX  REFDR3   
1  H2196_0001_ND3.JP2        94483  HRSC     MEX  REFDR3   

                     image_path                    label_path  \
0  data/hrsc/h1293_0000_nd3.jp2  data/hrsc/h1293_0000_nd3.lbl   
1  data/hrsc/h2196_0001_nd3.jp2  data/hrsc/h2196_0001_nd3.lbl   

  instrumentHostName                 instrumentName missionPhaseName  \
0       MARS EXPRESS  HIGH RESOLUTION STEREO CAMERA       MR_Phase_4   
1       MARS EXPRESS  HIGH RESOLUTION STEREO CAMERA       MR_Phase_7   

       idFromProvider observationMode instrumentId spacecraftId datasetId  \
0  H1293_0000_ND3.JP2          REFDR3         HRSC          MEX  MEX_HRSC   
1  H2196_0001_ND3.JP2          REFDR3         HRSC          MEX  MEX_HRSC   

  targetName  productCreationTime                   startTime  \
0       MARS  2016-03-04T20:31:26  2005-01-19T19:04:06.841000   
1       MARS  2016-03-14T11:38:04  2005-09-29T15:25:45.728001   

                     stopTime  
0  2005-01-19T19:09:29.847000  
1  2005-09-29T15:30:29.709999

In [14]:
from pandas import concat
from geopandas import GeoDataFrame

gdf = GeoDataFrame(concat([
        gdf_hrsc_new,
        gdf_ctx_new,
        gdf_hirise_new,
    ],ignore_index=True))

In [15]:
gdf

geometry Center_latitude  \
0  POLYGON ((-17.82000 40.06500, -17.81000 39.882...         29.7761   
1  POLYGON ((-16.59400 8.63200, -16.60100 8.78700...         16.2676   
2  POLYGON ((-17.29000 23.65040, -17.20000 22.936...         23.2688   
3  POLYGON ((-17.80000 27.00540, -17.13000 21.805...         24.3758   
4  POLYGON ((-17.43700 23.17200, -17.54100 23.161...         23.2452   
5  POLYGON ((-17.37900 22.11410, -17.48200 22.102...         22.2517   

  Center_longitude Easternmost_longitude Emission_angle  \
0         341.8979                342.89           None   
1         344.0262               345.891           None   
2         342.4958                 342.8           5.85   
3         342.2817                342.87           0.09   
4         342.5015               342.563       5.816564   
5          342.551               342.621       0.295027   

  Footprints_cross_meridian Incidence_angle Map_scale Maximum_latitude  \
0                     False            None      12.5           40.065   
1                     False            None        25          23.9027   
2                     False           48.88      5.66          23.6504   
3                     False           59.39      5.66          27.0054   
4                     False       48.900419      0.25          23.3293   
5                     False       58.737351      0.25          22.4007   

  Minimum_latitude            Observation_time Phase_angle  \
0           19.519  2005-01-19T19:06:48.344002        None   
1            8.632  2005-09-29T15:28:07.719000        None   
2          22.8866  2006-11-12T15:12:52.777000       43.11   
3          21.7462  2007-02-18T05:52:00.318000       59.41   
4           23.161  2006-11-12T15:12:52.248001   43.137054   
5          22.1027  2007-02-18T05:51:20.181000   58.835998   

  Product_creation_time Solar_longitude Target_name  \
0   2016-03-04T20:31:26            None        MARS   
1   2016-03-14T11:38:04            None        MARS   
2   2007-05-18T22:07:23          134.36        MARS   
3   2007-08-09T21:43:46          185.96        MARS   
4   2009-09-22T22:45:09       134.35984        MARS   
5   2009-11-11T10:51:58      185.953173        MARS   

               UTC_start_time               UTC_stop_time  \
0  2005-01-19T19:04:06.841000  2005-01-19T19:09:29.847000   
1  2005-09-29T15:25:45.728001  2005-09-29T15:30:29.709999   
2  2006-11-12T15:12:46.050999  2006-11-12T15:12:59.504002   
3  2007-02-18T05:51:11.313000  2007-02-18T05:52:49.324001   
4  2006-11-12T15:12:50.716999  2006-11-12T15:12:53.779999   
5  2007-02-18T05:51:17.450001  2007-02-18T05:51:22.912001   

  Westernmost_longitude                                         browse_url  \
0               340.677  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   
1               342.568  https://pds-geosciences.wustl.edu/mex/mex-m-hr...   
2                342.19                                               None   
3                341.66                                               None   
4               342.439  https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/...   
5               342.481  https://hirise.lpl.arizona.edu/PDS/EXTRAS/RDR/...   

                           id image_kbytes    inst mission    type  \
0          H1293_0000_ND3.JP2       286318    HRSC     MEX  REFDR3   
1          H2196_0001_ND3.JP2        94483    HRSC     MEX  REFDR3   
2  P01_001388_2034_XI_23N017W        35001     CTX     MRO     EDR   
3  P04_002641_2046_XN_24N017W       252001     CTX     MRO     EDR   
4         PSP_001388_2035_RED       373388  HIRISE     MRO  RDRV11   
5         PSP_002641_2025_RED       980973  HIRISE     MRO  RDRV11   

                                image_path  \
0             data/hrsc/h1293_0000_nd3.jp2   
1             data/hrsc/h2196_0001_nd3.jp2   
2  data/ctx/P01_001388_2034_XI_23N017W.IMG   
3  data/ctx/P04_002641_2046_XN_24N017W.IMG   
4      data/hirise/PSP_001388_2035_RED.JP2   
5      data/hirise/PSP_002641_2025_RED.JP2 

In [16]:
gdf.to_file('Mawrth_Vallis_downloaded_sample_products.geojson', driver='GeoJSON', index=False)